In [1]:
import numpy as np
import pandas as pd
import os
import glob
import sys


In [6]:
DATA_DIR = './../../generated_data'
DIR = 'peru_export'
id_col = 'PanjivaRecordID'

In [ ]:
# Set up an indexing scheme where each of the entities of each column is given a separate id
# Join the train & test sets
# For each column take the set of entities and "enumerate" them.


In [28]:
train_data_file = os.path.join( DATA_DIR, DIR, 'train_data.csv' )
test_data_file = os.path.join( DATA_DIR, DIR, 'test_data.csv' )
anomaly_data_file_c1 = os.path.join( DATA_DIR, DIR, 'anomalies_c1_data.csv' )
anomaly_data_file_c2 = os.path.join( DATA_DIR, DIR, 'anomalies_c2_data.csv' )
anomaly_data_file_c3 = os.path.join( DATA_DIR, DIR, 'anomalies_c3_data.csv' )

train_df = pd.read_csv(train_data_file)
test_df = pd.read_csv(test_data_file)
anomaly_df_c1 = pd.read_csv(anomaly_data_file_c1)
anomaly_df_c2 = pd.read_csv(anomaly_data_file_c2)
anomaly_df_c3 = pd.read_csv(anomaly_data_file_c3)

target_dfs = [ train_df, test_df, anomaly_df_c1, anomaly_df_c2, anomaly_df_c3 ]

In [29]:
feature_cols = list(train_df.columns)
feature_cols.remove(id_col)


In [30]:
feature_col2id = { e[0]:e[1] for e in enumerate(feature_cols,0)}

In [9]:
# add _<col_id> to all the training data

{0: 'CustomsCode',
 1: 'LocationCode',
 2: 'PortOfUnladingUNLOCODE',
 3: 'ShipmentDestination',
 4: 'ShipperPanjivaID',
 5: 'TransportMethod',
 6: 'hscode_6'}

## Convert the dataframe data to id s

In [33]:
def get_entity_map(train_df, feature_cols, target_dfs):
    
    feature_col2id = { e[1]:e[0] for e in enumerate(feature_cols,0)}
    ref_df = pd.DataFrame(train_df, copy=True)
    
    # ----- Auxillary functions ------- #
    def _append_colId(row, col, feature_col2id):
        return str(row[col]) + '_' + str(feature_col2id[col])
    
    def replace_with_entity_id(row,col,entity2id_map):
        v = row[col]
        return entity2id_map[v]
    # -------------------------------- #
    
    for col in feature_cols:
        ref_df[col] = ref_df.apply(_append_colId,axis=1,args=(col, feature_col2id,))
    
    # Create a set of all entities
    all_entities = []
    for col in feature_cols:
        _tmp = list(sorted(set(ref_df[col])))
        all_entities.extend(_tmp)
        pass
    entity2id_map = {e[1]:e[0] for e in enumerate(all_entities,0) }
    
    # ------------------------------------------ #
    # Convert the entities in train and test sets
    for _df in target_dfs:
        for col in feature_cols:
            _df[col] = _df.apply(
                _append_colId,
                axis=1,
                args=(col,feature_col2id,)
            )
             
            _df[col] = _df.apply(
                replace_with_entity_id,
                axis=1,
                args=(col,entity2id_map,)
            )
        
    return target_dfs   

In [34]:
result = get_entity_map(train_df, feature_cols, target_dfs)
train_df = result[0]
test_df =  result[1] 
anomaly_df_c1 =  result[2] 
anomaly_df_c2 =  result[3] 
anomaly_df_c3 =  result[4]

## Set up text files for the matlab input


In [38]:
np.savetxt('np.txt', train_df.values, fmt='%d')